In [5]:
#r "nuget:Apache.Arrow"

Installed Packages Apache.Arrow, 11.0.0

In [6]:
using System;
using System.IO;
using System.Threading.Tasks;
using Apache.Arrow;
using Apache.Arrow.Ipc;
using Apache.Arrow.Types;

In [7]:
void CreateRandomFile(string path)
{
            // Generate random data
        var random = new Random();
        int numRows = 10;

        var names = new string[] { "Alice", "Bob", "Charlie", "David", "Eve", "Frank", "Grace", "Hank", "Iris", "John" };
        var nameBuilder = new StringArray.Builder();
        var valueBuilder = new FloatArray.Builder();

        for (int i = 0; i < numRows; i++)
        {
            nameBuilder.Append(names[random.Next(names.Length)]);
            valueBuilder.Append((float)random.NextDouble() * 100);
        }

        // Create a RecordBatch with the generated data
        var schema = new Schema.Builder()
            .Field(f => f.Name("name").DataType(StringType.Default))
            .Field(f => f.Name("value").DataType(FloatType.Default))
            .Build();

        var recordBatch = new RecordBatch(
            schema,
            new IArrowArray[] { nameBuilder.Build(), valueBuilder.Build() },
            numRows
        );

        // Write the RecordBatch to an Arrow IPC file
        using var stream = File.OpenWrite(path);
        using var writer = new ArrowStreamWriter(stream, schema);
        
        writer.WriteRecordBatch(recordBatch);

        Console.WriteLine($"Random Arrow IPC file created at: {path}");
}

In [8]:
public IEnumerable<object> ReadFile(string path)
{
        // Read the Arrow IPC file into a RecordBatch
        RecordBatch recordBatch;
        using (var stream = File.OpenRead(path))
        {
            var reader = new ArrowStreamReader(stream);
            recordBatch = reader.ReadNextRecordBatch();
        }

        // Access columns by name
        StringArray nameColumn = recordBatch.Column(recordBatch.Schema.GetFieldIndex("name")) as StringArray;
        FloatArray valueColumn = recordBatch.Column(recordBatch.Schema.GetFieldIndex("value")) as FloatArray;

        // Print the contents of the columns
        var ret = new List<object>();
        for (int i = 0; i < recordBatch.Length; i++)
        {
            ret.Add((Name: nameColumn.GetString(i), Value: valueColumn.GetValue(i)));
        }
        return ret;
}

In [16]:
var path = "/home/aron/ArrowPlayground/random_arrow_ipc_file.arrow";
CreateRandomFile(path);
ReadFile(path)

Error: System.IO.DirectoryNotFoundException: Could not find a part of the path '/home/arond/ArrowPlayground/random_arrow_ipc_file.arrow'.
   at Interop.ThrowExceptionForIoErrno(ErrorInfo errorInfo, String path, Boolean isDirError)
   at Microsoft.Win32.SafeHandles.SafeFileHandle.Open(String fullPath, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, UnixFileMode openPermissions, Int64& fileLength, UnixFileMode& filePermissions, Func`4 createOpenException)
   at System.IO.Strategies.OSFileStreamStrategy..ctor(String path, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, Nullable`1 unixCreateMode)
   at System.IO.File.OpenWrite(String path)
   at Submission#7.CreateRandomFile(String path)
   at Submission#16.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 pre